In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
from google.colab import userdata
import os

In [ ]:
# install transformer to get the trasnformer library in hugging face
# install torch since working with images (the PyTorch deep learning framework), and  Pillow for opening and editing images
# tqdm for progress bars
# datasets (Hugging Face library) tool that helps you load existing datasets, create your own datasets, Work efficiently with large data  without running out of memory
!pip install transformers datasets torch torchvision pillow tqdm



Using ViT Base pretrained on ImageNet as the first pretrained model for feature extraction

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModel
import pickle
from tqdm import tqdm


In [ ]:
#Preprocessing the data so it can be used by the ViT Base
def preprocess_bands(row, band1, band2):
  band1 = np.array(row[band1], dtype = np.float32)
  band2 = np.array(row[band2], dtype = np.float32)
  # stacking into 3 channels, since the pretrained model needs 3 channels
  img_array = np.stack([band1, band2, band1], axis = 2)
  img = Image.fromarray((img_array * 255).astype(np.uint8))
  return img

In [ ]:
#Preprocessing to be used by PyTorch:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [ ]:
# Making a Dataset class
class IcebergDataset(Dataset):
    def __init__(self, df, preprocess, band1, band2):
        self.df = df
        self.preprocess = preprocess  # transforms.Compose
        self.band1 = band1
        self.band2 = band2

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Preprocess bands into 3-channel PIL image
        band1 = np.array(row[self.band1], dtype=np.float32)
        band2 = np.array(row[self.band2], dtype=np.float32)
        img_array = np.stack([band1, band2, band1], axis=2)  # H x W x 3
        img = Image.fromarray((img_array * 255).astype(np.uint8))

        # Apply timm / torchvision transforms
        img_tensor = self.preprocess(img)  # <-- just call the transform

        label = torch.tensor(row['is_iceberg'], dtype=torch.float32)
        return img_tensor, label


In [ ]:
#from huggingface_hub import login
#login()


In [ ]:
!pip install timm
import timm
import torch
from PIL import Image
from torchvision import transforms
import numpy as np

In [ ]:
#Loading ViT Base

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.eval()
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [ ]:
masked_training_df = pd.read_json("masked_training.json")
print(masked_training_df.columns)

ssf_training_df = pd.read_json("ssf_training.json")
print(ssf_training_df.columns)

#zoom_training_df = pd.read_json("zoom_training.json")
#print(zoom_training_df.columns)


Index(['id', 'band_1', 'band_2', 'inc_angle', 'is_iceberg', 'hv_hh_ratio',
       'total_power', 'ratio_mean', 'power_mean', 'band_1_mean', 'band_2_mean',
       'masked_band1', 'masked_band2'],
      dtype='object')
Index(['id', 'inc_angle', 'is_iceberg', 'ratio_mean', 'power_mean',
       'band_1_mean', 'band_2_mean', 'band_1_ssf', 'band_2_ssf',
       'hv_hh_ratio_ssf'],
      dtype='object')


In [ ]:
#Create data loader for the different preprocessed data

dataset = IcebergDataset(masked_training_df, preprocess, 'band_1', 'band_2')
loader = DataLoader(dataset, batch_size=16, shuffle=False)

dataset_masked = IcebergDataset(masked_training_df, preprocess, 'masked_band1', 'masked_band2')
loader_masked = DataLoader(dataset_masked, batch_size=16, shuffle=False)

dataset_ssf = IcebergDataset(ssf_training_df, preprocess, 'band_1_ssf', 'band_2_ssf')
loader_ssf = DataLoader(dataset_ssf, batch_size=16, shuffle=False)

#dataset_zoom = IcebergDataset(zoom_training_df, preprocess, 'zoom_band1', 'zoom_band2')
#loader_zoom = DataLoader(dataset_zoom, batch_size=16, shuffle=False)


In [ ]:
def get_embedding(loader, filename, device = device):
  all_features = []
  all_labels = []

  for images, labels in tqdm(loader):
      images = images.to(device)
      with torch.no_grad():
          # Use timm's forward_features to get embeddings
          features = model.forward_features(images)  # shape [B, feature_dim]

      all_features.append(features.cpu().numpy())
      all_labels.append(labels.numpy())

  # Concatenate all batches
  all_features = np.concatenate(all_features, axis=0)
  all_labels = np.concatenate(all_labels, axis=0)

  # Save features to pickle
  with open(f"{filename}_ViT_features.pkl", "wb") as f:
      pickle.dump({"features": all_features, "labels": all_labels}, f)

  print(f"Feature extraction complete!{filename}_ViT_features.pkl")

In [ ]:
#get_embedding(loader_zoom, device = device)
get_embedding(loader,"training", device = device)
get_embedding(loader_masked, "masked_training", device = device)
get_embedding(loader_ssf,"ssf_training", device = device)

100%|██████████| 92/92 [16:25<00:00, 10.71s/it]


Feature extraction complete!training_ViT_features.pkl


100%|██████████| 92/92 [15:43<00:00, 10.26s/it]


Feature extraction complete!masked_training_ViT_features.pkl


100%|██████████| 92/92 [15:50<00:00, 10.33s/it]


Feature extraction complete!ssf_training_ViT_features.pkl


swin-tiny-patch4-window7-224-finetuned-sar model


*  Swin Transformers are Vision Transformers (ViTs) that use a hierarchical, windowed self-attention mechanism.

*  “Tiny” = small version, efficient for faster training/inference.

*  Patch size 4, window size 7 = defines how the image is split for attention computations.

*  Input image size = 224×224 (standard for ImageNet pretraining).

*  Fine-tuned on SAR backscatter patterns for icebergs vs. ships.



In [ ]:
from transformers import AutoImageProcessor, AutoModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoImageProcessor.from_pretrained("elifm/swin-tiny-patch4-window7-224-finetuned-sar")
model = AutoModel.from_pretrained("elifm/swin-tiny-patch4-window7-224-finetuned-sar")
model.eval()
model.to(device)

preprocessor_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/110M [00:00<?, ?B/s]

SwinModel(
  (embeddings): SwinEmbeddings(
    (patch_embeddings): SwinPatchEmbeddings(
      (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): SwinEncoder(
    (layers): ModuleList(
      (0): SwinStage(
        (blocks): ModuleList(
          (0): SwinLayer(
            (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attention): SwinAttention(
              (self): SwinSelfAttention(
                (query): Linear(in_features=96, out_features=96, bias=True)
                (key): Linear(in_features=96, out_features=96, bias=True)
                (value): Linear(in_features=96, out_features=96, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): SwinSelfOutput(
                (dense): Linear(in_features=96, out_features=96, bias=True)
            

In [ ]:
#Loading Pretrained Swin Transformer
model_name2 = "elifm/swin-tiny-patch4-window7-224-finetuned-sar"
swin_processor = AutoImageProcessor.from_pretrained(model_name2) # resizes, normalizes, and converts
swin_model = AutoModel.from_pretrained(model_name2)
swin_model.eval() # don't update weights/no gradient updates

# Move to GPU
swin_model.to(device)

model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

SwinModel(
  (embeddings): SwinEmbeddings(
    (patch_embeddings): SwinPatchEmbeddings(
      (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): SwinEncoder(
    (layers): ModuleList(
      (0): SwinStage(
        (blocks): ModuleList(
          (0): SwinLayer(
            (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attention): SwinAttention(
              (self): SwinSelfAttention(
                (query): Linear(in_features=96, out_features=96, bias=True)
                (key): Linear(in_features=96, out_features=96, bias=True)
                (value): Linear(in_features=96, out_features=96, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): SwinSelfOutput(
                (dense): Linear(in_features=96, out_features=96, bias=True)
            

In [ ]:
def get_embedding_swin(df, band1_col, band2_col, processor, model, device, dataset_name):
    all_features = []
    all_labels = []

    print("Moving model to device and setting eval mode")
    model.eval()
    model.to(device)

    for i in tqdm(range(len(df)), desc="Processing images"):
        try:
            print(f"\n🔹 Processing image {i+1}/{len(df)}")
            # Select 1 image/row
            row = df.iloc[i]

            # Convert bands to 3-channel PIL image
            img = preprocess_bands(row, band1_col, band2_col)
            print(f"  Image shape after preprocessing: {np.array(img).shape}")

            # Process image (resizes to model's expected input size,
            # normalizes pixel values, and coverts Python-image-library (PIL)
            # image to PyTorch tensor (multidimensional-array))
            # Why? machine learning models require Tensor, not an img
            inputs = processor(images=img, return_tensors="pt").to(device)
            print(f"  Input keys: {list(inputs.keys())}")
            print(f"  Input tensor shape: {inputs['pixel_values'].shape}")

            # Forward pass - Get embeddings
            with torch.no_grad():
                outputs = model(**inputs)
            print(f"  Outputs type: {type(outputs)}")

            # Safely handle model outputs
            if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
                features = outputs.pooler_output.cpu().numpy()
                print(f"  Using pooler_output, shape: {features.shape}")
            else:
                features = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                print(f"  Using mean of last_hidden_state, shape: {features.shape}")

            # extract embeddings
            all_features.append(features)
            all_labels.append(row["is_iceberg"])

        except Exception as e:
            print(f"Error processing image {i}: {e}")

    # Combine all embeddings and labels
    all_features = np.vstack(all_features)
    all_labels = np.array(all_labels)
    print(f"\nAll features stacked, shape: {all_features.shape}")
    print(f"All labels stacked, shape: {all_labels.shape}")

    # Save to pickle
    with open(f"{dataset_name}_swin_features.pkl", "wb") as f:
        pickle.dump({"features": all_features, "labels": all_labels}, f)
    print(f"Swin feature extraction complete! Saved to {dataset_name}_swin_features.pkl")


In [ ]:
# Call extract features
#get_embedding_swin(masked_training_df, 'band_1', 'band_2', processor, model, device, "training_df")
get_embedding_swin(masked_training_df, 'masked_band1', 'masked_band2', processor, model, device, "masked_training_df")
get_embedding_swin(ssf_training_df, 'band_1_ssf', 'band_2_ssf', processor, model, device, "ssf_training_df")
#get_embedding_swin(zoom_training_df, 'band_1', 'band_2', processor, model, device, "zoom_training_df")


Moving model to device and setting eval mode


Processing images:   0%|          | 0/1471 [00:00<?, ?it/s]


🔹 Processing image 1/1471
  Image shape after preprocessing: (75, 75, 3)
  Input keys: ['pixel_values']
  Input tensor shape: torch.Size([1, 3, 224, 224])


Processing images:   0%|          | 1/1471 [00:00<09:32,  2.57it/s]

  Outputs type: <class 'transformers.models.swin.modeling_swin.SwinModelOutput'>
  Using pooler_output, shape: (1, 768)

🔹 Processing image 2/1471
  Image shape after preprocessing: (75, 75, 3)
  Input keys: ['pixel_values']
  Input tensor shape: torch.Size([1, 3, 224, 224])


...

  Outputs type: <class 'transformers.models.swin.modeling_swin.SwinModelOutput'>
  Using pooler_output, shape: (1, 768)

🔹 Processing image 1470/1471
  Image shape after preprocessing: (75, 75, 3)
  Input keys: ['pixel_values']
  Input tensor shape: torch.Size([1, 3, 224, 224])


Processing images: 100%|█████████▉| 1470/1471 [07:55<00:00,  3.31it/s]

  Outputs type: <class 'transformers.models.swin.modeling_swin.SwinModelOutput'>
  Using pooler_output, shape: (1, 768)

🔹 Processing image 1471/1471
  Image shape after preprocessing: (75, 75, 3)
  Input keys: ['pixel_values']
  Input tensor shape: torch.Size([1, 3, 224, 224])


Processing images: 100%|██████████| 1471/1471 [07:56<00:00,  3.09it/s]

  Outputs type: <class 'transformers.models.swin.modeling_swin.SwinModelOutput'>
  Using pooler_output, shape: (1, 768)

All features stacked, shape: (1471, 768)
All labels stacked, shape: (1471,)
Swin feature extraction complete! Saved to ssf_training_df_swin_features.pkl


Use a plain timm Swin Transformer

In [ ]:
device_swin_timm = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_swin_timm = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)
model_swin_timm.eval()
model_swin_timm.to(device)

In [ ]:
get_embedding(loader_masked, device=device_swin_timm)
get_embedding(loader_ssf, device=device_swin_timm)
get_embedding(loader_zoom, device=device_swin_timm)